In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time


import string

import re

import json

In [2]:
def mergeDictionary(dict_1, dict_2):
    dict_3 = {**dict_1, **dict_2}
    for key, value in dict_3.items():
        if key in dict_1 and key in dict_2:
            dict_3[key] = value + dict_1[key]
    return dict_3

In [3]:
import nltk
nltk.download('words')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package words to /home/cdsw/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [4]:

import tensorflow as tf
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"

model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [5]:
words = set(nltk.corpus.words.words())

In [6]:
def embed(input):
    return model(input)

In [7]:
def basic_stats_error(datalist):
    
    if len(datalist)>1:
        result_describe = list(describe(datalist)[2:])
        result_percentile = result_describe + list(np.percentile(datalist,[10,25,50,75,90]))
        array = np.array(datalist)
        result_percentile = result_percentile+[len(array),len(array[np.where(array>0)[0]])/len(array)*100,
                                               len(array[np.where(array<0)[0]])/len(array)*100]
    else:
        result_percentile = []

    return result_percentile

In [8]:
def df_lambda_detect_not_words(df, column_name, new_column):
    Lem = WordNetLemmatizer()
    df[new_column] = df[column_name].apply(lambda x: " ".join(w for w in nltk.wordpunct_tokenize(x) \
         if Lem.lemmatize(w.lower()) not in words))
    return df

In [9]:
def df_lambda_remove_punctuation(df, column_name, new_column):
    df[new_column] = df[column_name].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    return df

In [10]:
def df_lambda_replace_characters(df, column_name, new_column, dictionary):
    df[new_column] = df[column_name].apply(lambda x: replace_words(x, dictionary))       
    return df


In [11]:
def replace_words(strtext, dictionary):
    pattern = re.compile(r'(?<!\w)(' + '|'.join(re.escape(key) for key in dictionary.keys()) + r')(?!\w)')
    result = pattern.sub(lambda x: dictionary[x.group()], strtext)
    return result

In [12]:
with open("replacement_dictionary_ordered.json") as json_file:
    replacement_dictionary_2 = json.load(json_file)


In [13]:
import glob
nounsentence_files = glob.glob("comments/raw_comments/*_filtered_noun_sentences.fea")

In [14]:
nounsentence_files[8:9]

['comments/raw_comments/agent_comments_FL_filtered_noun_sentences.fea']

In [ ]:
for file in nounsentence_files[8:9]:
    
    print(file)
    noun_sentence_df = pd.read_feather(file)
    print(len(noun_sentence_df))
    tic = time.time()
    
    noun_sentence_df_20 = noun_sentence_df[noun_sentence_df['count']>20]
    
    noun_sentence_df_20 = df_lambda_remove_punctuation(noun_sentence_df_20, 'noun_sentence', 'noun_sentence_no_punct')
    
    noun_sentence_df_20 = df_lambda_replace_characters(noun_sentence_df_20,  'noun_sentence_no_punct', 
                                                   'noun_sentence_corrections', replacement_dictionary_2)
    
    import time
    columns = ['noun_sentence_corrections']
    for column in columns:
        print(column)
        tic = time.time()
        new_column = column + '_embeddings'
        noun_sentence_df_20[new_column] = noun_sentence_df_20[column].apply(lambda x: embed([x]).numpy()[0]) 
        toc = time.time()
        print(toc - tic)
        
    noun_sentence_df_1000 = noun_sentence_df_20[noun_sentence_df_20['count']>1000]
    noun_sentence_df_1000_corr = noun_sentence_df_1000[['noun_sentence_corrections',
                                                        'noun_sentence_corrections_embeddings']]
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.set_index('noun_sentence_corrections')
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.reset_index()
    
    column = 'noun_sentence_corrections_embeddings'
    nelements = (len(noun_sentence_df_1000_corr[column][0]))
    #print(nelements)
    list_tmp = [str(i) for i in range(nelements)] 
    column_names = [column + '_' + str(i) for i in list_tmp] 
    #print(column_names)
    split_df = pd.DataFrame(noun_sentence_df_1000_corr[column].tolist(), columns=column_names)
    
    noun_sentence_df_1000_corr = pd.concat([noun_sentence_df_1000_corr, split_df], axis=1)
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.drop(columns = [column])
    
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.set_index('noun_sentence_corrections')
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.T
    
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.reset_index()
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.drop(columns=['index'])
    
    correlations_df = noun_sentence_df_1000_corr.corr(method='pearson')
    
    skill_group = {}
    test_df = correlations_df.copy('deep')

    while len(test_df) > 0:
        idd = test_df.index[0]
        print(idd)
        group = list(np.array(test_df.loc[idd][test_df.loc[idd]>0.8].index))
        print(len(group))
        #print(noun_sentence_df_20_reindex.loc[group,'AVM_Error_Average'])
        skill_group[group[0]] = group
        test_df = test_df.drop(labels = group, axis = 1)
        test_df = test_df.drop(labels = group, axis = 0)

    noun_sentence_df_50_1000 = noun_sentence_df_20[(noun_sentence_df_20['count']>50)
                                                   &(noun_sentence_df_20['count']<=1000)]
    
    noun_sentence_df_50_1000_reindex = noun_sentence_df_50_1000.set_index('noun_sentence_corrections')
    
    noun_sentence_df_50_1000_reindex = noun_sentence_df_50_1000_reindex.loc[:,['noun_sentence_corrections_embeddings']]
    
    noun_sentence_df_50_1000_reindex = noun_sentence_df_50_1000_reindex.reset_index()

    column = 'noun_sentence_corrections_embeddings'
    nelements = (len(noun_sentence_df_50_1000_reindex[column][0]))
    #print(nelements)
    list_tmp = [str(i) for i in range(nelements)] 
    column_names = [column + '_' + str(i) for i in list_tmp] 
    #print(column_names)
    split_df = pd.DataFrame(noun_sentence_df_50_1000_reindex[column].tolist(), columns=column_names)
    
    noun_sentence_df_50_1000_corr = pd.concat([noun_sentence_df_50_1000_reindex, split_df], axis=1)
    noun_sentence_df_50_1000_corr = noun_sentence_df_50_1000_corr.drop(columns = [column])
    
    noun_sentence_df_50_1000_corr = noun_sentence_df_50_1000_corr.set_index('noun_sentence_corrections')
    noun_sentence_df_50_1000_corr = noun_sentence_df_50_1000_corr.T
    
    noun_sentence_df_50_1000_corr = noun_sentence_df_50_1000_corr.reset_index()
    
    noun_sentence_df_1000_reindex = noun_sentence_df_1000.set_index('noun_sentence_corrections')
    
    columns = ['AVM_Error_List','Listing_Error_List',
          'Listingid_List','noun_sentence_corrections_embeddings']
    noun_sentence_df_1000_reindex_df = pd.DataFrame(columns=columns)
    
    for key in skill_group:
        #print(key)
        #print(skill_group[key])
        if len(skill_group[key])==1:
            #print(noun_sentence_df_1000_reindex.loc[key,columns])
            noun_sentence_df_1000_reindex_df = noun_sentence_df_1000_reindex_df.append(\
                                                        noun_sentence_df_1000_reindex.loc[key,columns])

        else:
            print(skill_group[key])
            temp_df = pd.DataFrame(columns=columns)
            temp_df =  temp_df.append(noun_sentence_df_1000_reindex.loc[skill_group[key],columns].iloc[0])
            for ii in range(3):
                temp_df[noun_sentence_df_1000_reindex.loc[skill_group[key],columns].columns[ii]] = \
                                           [np.concatenate(noun_sentence_df_1000_reindex.loc[skill_group[key],columns]\
                                               [noun_sentence_df_1000_reindex.loc[skill_group[key],columns].columns[ii]].values)]
            noun_sentence_df_1000_reindex_df = noun_sentence_df_1000_reindex_df.append(temp_df.loc[:,columns])
            subgroup = skill_group[key][1:]
    
    noun_sentence_df_1000_corr = noun_sentence_df_1000_reindex_df[['noun_sentence_corrections_embeddings']]
    
    column = 'noun_sentence_corrections_embeddings'
    nelements = (len(noun_sentence_df_1000_corr[column][0]))
    #print(nelements)
    list_tmp = [str(i) for i in range(nelements)] 
    column_names = [column + '_' + str(i) for i in list_tmp] 
    #print(column_names)
    split_df = pd.DataFrame(noun_sentence_df_1000_corr[column].tolist(), columns=column_names)
    
    noun_sentence_df_1000_corr = pd.concat([noun_sentence_df_1000_corr.reset_index(), split_df], axis=1)
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.drop(columns = [column]).set_index('index')

    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.T
    
    noun_sentence_df_1000_corr = noun_sentence_df_1000_corr.reset_index().drop(columns=['index'])
    
    noun_sentence_df_50_1000_corr = noun_sentence_df_50_1000_corr.rename(columns={'noun_sentence_corrections':'index'})
    
    noun_sentence_df_50_1000_corr.index.rename('index')
    
    correlations_50_1000_df = noun_sentence_df_50_1000_corr.corr()
    
    skill_group_50_1000 = {}
    test_df = correlations_50_1000_df.copy('deep')

    while len(test_df) > 0:
        idd = test_df.index[0]
        print(idd)
        print(len(test_df))
        group = list(np.array(test_df.loc[idd][test_df.loc[idd]>0.8].index))
        print(len(group))
        skill_group_50_1000[group[0]] = group
        test_df = test_df.drop(labels = group, axis = 1)
        test_df = test_df.drop(labels = group, axis = 0)
        
    noun_sentence_df_50_1000_reindex = noun_sentence_df_50_1000.set_index('noun_sentence_corrections')
    
    columns = ['AVM_Error_List','Listing_Error_List',
          'Listingid_List','noun_sentence_corrections_embeddings']
    noun_sentence_df_50_1000_reindex_df = pd.DataFrame(columns=columns)
    
    
    for key in skill_group_50_1000:
        print(key)
        #print(skill_group[key])
        if len(skill_group_50_1000[key])==1:
            #print(noun_sentence_df_1000_reindex.loc[key,columns])
            noun_sentence_df_50_1000_reindex_df = noun_sentence_df_50_1000_reindex_df.append(\
                                                        noun_sentence_df_50_1000_reindex.loc[key,columns])

        else:
            #print(skill_group_50_1000[key])
            temp_df = pd.DataFrame(columns=columns)
            temp_df =  temp_df.append(noun_sentence_df_50_1000_reindex.loc[[skill_group_50_1000[key][0]],columns].iloc[[0]])
            for ii in range(3):
                temp_df[noun_sentence_df_50_1000_reindex.loc[skill_group_50_1000[key],columns].columns[ii]] = \
                                           [np.concatenate(noun_sentence_df_50_1000_reindex.loc[skill_group_50_1000[key],columns]\
                                               [noun_sentence_df_50_1000_reindex.loc[skill_group_50_1000[key],columns].columns[ii]].values)]
            noun_sentence_df_50_1000_reindex_df = noun_sentence_df_50_1000_reindex_df.append(temp_df.loc[:,columns])
            subgroup = skill_group_50_1000[key][1:]
            
    noun_sentence_df_full_1000_reindex_df = noun_sentence_df_1000_reindex_df.append(noun_sentence_df_50_1000_reindex_df)
        
    skill_group_full = mergeDictionary(skill_group, skill_group_50_1000)

    noun_sentence_df_full_1000_corr = noun_sentence_df_full_1000_reindex_df[['noun_sentence_corrections_embeddings']]

    column = 'noun_sentence_corrections_embeddings'
    nelements = (len(noun_sentence_df_full_1000_corr[column][0]))
    #print(nelements)
    list_tmp = [str(i) for i in range(nelements)] 
    column_names = [column + '_' + str(i) for i in list_tmp] 
    #print(column_names)
    split_df = pd.DataFrame(noun_sentence_df_full_1000_corr[column].tolist(), columns=column_names)

    noun_sentence_df_full_1000_corr = pd.concat([noun_sentence_df_full_1000_corr.reset_index(), split_df], axis=1)
    noun_sentence_df_full_1000_corr = noun_sentence_df_full_1000_corr.drop(columns = [column]).set_index('index')
    
    noun_sentence_df_full_1000_corr = noun_sentence_df_full_1000_corr.T
    
    noun_sentence_df_full_1000_corr = noun_sentence_df_full_1000_corr.reset_index().drop(columns=['index'])
    
    correlations_full_1000_df = noun_sentence_df_full_1000_corr.corr()
    
    skill_group_full_1000 = {}
    test_df = correlations_full_1000_df.copy('deep')

    while len(test_df) > 0:
        idd = test_df.index[0]
        print(idd)
        print(len(test_df))
        group = list(np.array(test_df.loc[idd][test_df.loc[idd]>0.8].index))
        print(len(group))
        skill_group_full_1000[group[0]] = group
        test_df = test_df.drop(labels = group, axis = 1)
        test_df = test_df.drop(labels = group, axis = 0)
        
    skill_group_full_final = skill_group_full_1000.copy()

    for key in skill_group_full_1000:
        list_tmp = skill_group_full[key]
        skill_group_full_final[key] = skill_group_full_1000[key] + list_tmp
        
    columns = ['AVM_Error_List','Listing_Error_List',
          'Listingid_List','noun_sentence_corrections_embeddings']
    noun_sentence_df_final_reindex_df = pd.DataFrame(columns=columns)
    
    
    for key in skill_group_full_1000:
        print(key)
        #print(skill_group[key])
        if len(skill_group_full_1000[key])==1:
            #print(noun_sentence_df_1000_reindex.loc[key,columns])
            noun_sentence_df_final_reindex_df = noun_sentence_df_final_reindex_df.append(\
                                                        noun_sentence_df_full_1000_reindex_df.loc[key])

        else:
            #print(skill_group_50_1000[key])
            temp_df = pd.DataFrame(columns=columns)
            temp_df =  temp_df.append(noun_sentence_df_full_1000_reindex_df.loc[[skill_group_full_1000[key][0]]].iloc[[0]])
            for ii in range(3):
                temp_df[noun_sentence_df_full_1000_reindex_df.loc[skill_group_full_1000[key]].columns[ii]] = \
                                   [np.concatenate(noun_sentence_df_full_1000_reindex_df.loc[skill_group_full_1000[key]]\
                                   [noun_sentence_df_full_1000_reindex_df.loc[skill_group_full_1000[key]].columns[ii]].values)]
            noun_sentence_df_final_reindex_df = noun_sentence_df_final_reindex_df.append(temp_df)
            
    noun_file = file.split('.')[0] + '_noun_sentences_df_final_reindex.fea'
    noun_sentence_df_final_reindex_df.reset_index().to_feather(noun_file)
    
    import json
    # create json object from dictionary
    json = json.dumps(skill_group_full_final, indent = 4)
    f = open(noun_file+"noun_phrases_full_final.json","w")
    f.write(json)
    f.close()


comments/raw_comments/agent_comments_FL_filtered_noun_sentences.fea
6315307


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


noun_sentence_corrections


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


383.22973108291626
new doors
2
white shaker cabinets
2
new bathrooms
2
top bottom
1
new insulation
2
new tankless
1
new tub
1
new baths
1
new interior doors
1
new impact windows
2
pole barn
1
new baseboards
1
seminole heights
1
new driveway
1
modern conveniences
1
new vanities
1
new wood cabinets
2
new windows
1
new metal roof
2
new electrical panel
3
new double pane windows
2
subway tile
2
new cabinets
2
new kitchen
2
new lights
1
new vanity
1
new cabinetry
1
new fixtures
1
new duct work
1
luxury vinyl plank
2
new double
1
new floors
2
new deck
1
new privacy fence
2
new features
1
new hardware
1
new light fixtures
1
new irrigation system
2
amp service
1
new tile floors
4
new pavers
1
windows doors
1
new vinyl plank
1
new bathroom vanities
1
efficient windows
1
new countertops
1
bathrooms kitchen
3
new toilet
2
kitchen brand
1
new master bath
2
interior exterior paint
6
new dock
1
new sod
1
new ceramic tile
5
new fans
1
new window treatments
1
quartz countertops
3
new stainless steel a

...

neutral carpet
1
generous size
1
utility room
1
bath block home
1
culdesac street
1
s retreat
1
spacious floorplan
2
family room wood
1
famous beaches
1
large office
1
prime location
1
space home
1
solid block home
1
spacious bonus room
2
trey ceilings
1
marble floors
1
cbs home
1
glass doors lead
2
corian counter tops
3
separate sinks
1
diagonal tile
2
hobby room
1
bedroom floor plan
4
downstairs master bedroom
3
laundry room cabinets
1
soaker tub
1
world class
1
shower stall
1
side entry
1
family home
1
school district
1
large tile
1
easy commute tampa
1
backyard neighbors
2
bedroom design
1
great shape
1
excellent location
1
new tile roof
2
room pool
1
huge pantry
2
convenient location
1
pool spa
1
wall oven
1
ready occupancy
2
322 home
1
prep island
1
master bedroom bath
2
green space
1
seller contribute
1
sea world
1
picnic area
2
electric hurricane shutters
1
screen lanai
1
area attractions
1
rear neighbors
1
sliders lead
1
spacious family room
2
disney world
1
super location
1
l

['new doors', 'new french doors']
['white shaker cabinets', 'white cabinets']
['new bathrooms', 'new bathroom']
['new insulation', 'extra insulation']
['new impact windows', 'impact windows']
['new wood cabinets', 'wood cabinets']
['new metal roof', 'metal roof']
['new electrical panel', 'new electric panel', 'electrical panel']
['new double pane windows', 'double pane windows']
['subway tile', 'subway tile backsplash']
['new cabinets', 'new kitchen cabinets']
['new kitchen', 'new kitchen appliances']
['luxury vinyl plank', 'vinyl plank']
['new floors', 'new wood floors']
['new privacy fence', 'privacy fence']
['new irrigation system', 'irrigation system']
['new tile floors', 'new tile floor', 'new tile', 'tile floors']
['bathrooms kitchen', 'kitchen bathroom', 'kitchen bathrooms']
['new toilet', 'new toilets']
['new master bath', 'master bath']
['interior exterior paint', 'new interior exterior paint', 'interior exterior', 'fresh interior exterior paint', 'exterior paint', 'interior p

['beautiful wood', 'beautiful hardwood']
['open concept floor plan', 'open floor plan']
['beautiful laminate', 'beautiful wood laminate']
['perfect condition', 'excellent condition', 'great condition']
['huge pool', 'large pool']
['beautiful yard', 'beautiful backyard', 'nice yard']
['perfect spot', 'perfect place', 'perfect location']
['ready new owners', 'ready new owner']
['pool area', 'lanai pool area']
['friends family', 'family friends']
['glass entry door', 'glass door', 'glass front door', 'glass front doors', 'glass doors']
['amazing views', 'spectacular views', 'beautiful views', 'great views']
['tile floor', 'floor tile']
['main home', 'main house']
['reverse osmosis', 'reverse osmosis system']
['kitchen boasts', 'kitchen overlooks']
['ceramic tile kitchen', 'ceramic tile foyer']
['car garage pool home', 'car garage home']
['indoor laundry room', 'indoor laundry', 'interior laundry room']
['tons upgrades', 'lots upgrades']
['tile laminate', 'laminate tile']
['1car garage', '

['plenty counter space', 'lots counter space']
['downstairs master suite', 'upstairs master suite', 'master suite downstairs']
['side entry garage', 'car side entry garage']
['spacious foyer', 'large foyer', 'spacious home']
['huge master bath', 'large master bath']
['room fireplace', 'family room fireplace']
['closet pantry', 'pantry closet']
['room den', 'bedroom den', 'bedroom  den']
['seperate shower', 'separate walkin shower', 'separate shower', 'separate walk shower', 'separate tub shower']
['kitchen family room', 'family room kitchen', 'kitchen overlooks family room']
['private bath', 'private baths']
['fireplace family room', 'large family room fireplace']
['wonderful floor plan', 'great room floor plan', 'fantastic floor plan', 'great floor plan', 'nice floor plan']
['amazing opportunity', 'wonderful opportunity', 'incredible opportunity', 'fantastic opportunity', 'great opportunity', 'excellent opportunity']
['spacious florida room', 'large florida room', 'huge florida room']

highend fixtures
35471
2
entire home brand
35469
1
granite counters brand
35468
24
general contractor
35444
1
new waterproof vinyl plank
35443
2
ac ductwork
35441
2
fresh tile
35439
1
new appliances fixtures
35438
6
window casings
35432
1
new finishes
35431
2
turnkey investment
35429
1
tack room
35428
1
modern bathroom
35427
3
shaker style cabinetry
35424
4
complete transformation
35420
1
horses livestock
35419
4
refrigerator range
35415
4
new chefs kitchen
35411
2
old charm
35409
2
soffit fascia
35407
1
complete new kitchen
35406
2
professional stainless steel appliances
35404
2
horse stall
35402
3
new kitchen with granite counters
35399
19
new moldings
35380
1
new bathtub
35379
2
new water lines
35377
1
gorgeous bathrooms
35376
6
new white kitchen cabinets
35370
7
fresh sod
35363
1
new paint interior exterior
35362
11
new luxury plank
35351
1
dry acres
35350
1
doors drawers
35349
2
designer faucet
35347
2
new plush carpeting bedrooms
35345
4
new vinyl wood plank
35341
4
true lakefron

In [ ]:
noun_sentence_df_20.head(10)

In [ ]:
noun_sentence_df_50['AVM_Error_Indicators']

In [ ]:
noun_sentence_df_2000 = noun_sentence_df[noun_sentence_df['count']>2000]

In [ ]:
len(noun_sentence_df_2000)

In [ ]:

import seaborn as sns
sns.set()
def plot_similarity(labels, features, rotation):
    corr = np.inner(features, features)
    sns.set(font_scale=1.2)
    g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd")
    g.set_xticklabels(labels, rotation=rotation)
    g.set_title("Semantic Textual Similarity")

def run_and_plot(messages_):
    message_embeddings_ = model(messages_)
    plot_similarity(messages_, message_embeddings_, 90)

def plot_similarity_serie(labelref, labels, rotation):
    #print(labelref)
    #print(labels)
    message_embeddings_ = model(labels)
    print(message_embeddings_.shape)
    messageref_embeddings_ = model(labelref) 
    corr = []
    for i in range(len(labels)):
        corr.append(np.inner(message_embeddings_[i,:],messageref_embeddings_))
    
    plt.plot(range(len(labels)),corr)
    plt.xticks(range(len(labels)),labels, rotation=rotation)
    plt.title(labelref)
    return corr

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from scipy.stats import pearsonr
from gensim.models import Word2Vec

In [ ]:

def plot_similarity_serie_error(labelref, labels, errors, rotation):
    #print(labelref)
    #print(labels)
    message_embeddings_ = model(labels)
    print(message_embeddings_.shape)
    messageref_embeddings_ = model(labelref) 
    corr = np.zeros(len(labels))
    cossim = np.zeros(len(labels))
    worddistance = np.zeros(len(labels))
    
    labelrefwords = labelref[0].split()
    print(labelrefwords)
    
    for i in range(len(labels)):
        labelwords = labels[i].split()
        A = np.array(message_embeddings_[i,:])
        B = np.array(messageref_embeddings_)
        #print(A.shape,B.shape)
        corr[i] = pearsonr(A,B[0])[0]
        #cossim[i] = cosine_similarity(A.reshape(1,-1),B.reshape(1,-1))
        
        worddistance[i] = model_gensim.wmdistance(labelrefwords, labelwords)
        
    plt.plot(range(len(labels)),corr)
    plt.plot(range(len(labels)),worddistance)                                 
    
    for i, txt in enumerate(errors):
        plt.annotate(str(round(txt, 3)), xy =((range(len(labels)))[i], corr[i]), ha='center')
        
    plt.xticks(range(len(labels)),labels, rotation=rotation)
    plt.title(labelref)
    return corr, cossim, worddistance

In [ ]:
run_and_plot(list(noun_sentence_df_1000['noun_sentence'].values[:20]))

In [ ]:

fig = plt.figure(figsize=(15, 6))
plot_similarity_serie([noun_sentence_df_2000['noun_sentence'].values[0]],
      list(noun_sentence_df_2000['noun_sentence'].values[:40]),90)

In [ ]:
fig = plt.figure(figsize=(17, 6))
corr = plot_similarity_serie_error([noun_sentence_df_1000['noun_sentence'].values[5]],
                                    list(noun_sentence_df_1000['noun_sentence'].values[:40]),
                                    list(noun_sentence_df_1000['AVM_Error_Average'].values[:40]),90)
#print(np.array(corr).shape)
#plt.text(range(len(list(noun_sentence_df_2000['noun_sentence'].values[:40]))), 
#         corr, 
#         str(list(noun_sentence_df_2000['AVM_Error_Average'].values[:40])))

In [ ]:
corr[2]

In [ ]:
fig = plt.figure(figsize=(15, 6))
corr = plot_similarity_serie([noun_sentence_df_2000['noun_sentence'].values[10]],
      list(noun_sentence_df_2000['noun_sentence'].values[:40]),90)

In [ ]:
fig = plt.figure(figsize=(15, 6))
corr = plot_similarity_serie([noun_sentence_df_2000['noun_sentence'].values[30]],
      list(noun_sentence_df_2000['noun_sentence'].values[:40]),90)

In [ ]:
np.inner(np.array([0,1]),np.array([0,1]))

In [ ]:
list_of_noun_sentences

In [ ]:
list_of_embeddings

In [ ]:
sentence_possibilities = noun_sentence_df_1000['noun_sentence'].values

In [ ]:
sentence_possibilities

In [ ]:

keys_list = list(noun_sentence_df['noun_sentence'].values)
values_list = [[] for i in range(len(keys_list))]
noun_sentences_dictionary = dict(zip(keys_list, values_list))

In [ ]:
list_of_noun_sentences = []
list_of_noun_sentences.append(noun_sentence_df_1000['noun_sentence'].values[0])

list_of_embeddings = []
list_of_embeddings.append(noun_sentence_df_1000['noun_sentence_embeddings'].values[0])

for idd in range(len(noun_sentence_df_1000)):
#for idd in range(50):
    if (idd%30==0): print(idd,len(noun_sentence_df_1000))
    labelrefwords = noun_sentence_df_1000['noun_sentence'].values[idd].split()
    corrs = np.zeros(len(list_of_noun_sentences))
    #distances = np.zeros(len(list_of_noun_sentences))
    #print(corrs)
    for jdd in range(len(list_of_noun_sentences)):
        labelwords = list_of_noun_sentences[jdd].split()
        corrs[jdd] = pearsonr(noun_sentence_df_1000['noun_sentence_embeddings'].values[idd],
                      list_of_embeddings[jdd])[0]
        #distances[jdd] = model_gensim.wmdistance(labelrefwords, labelwords)
    #print(corrs)
    #print((corrs<0.79) & (distances>0.5))
    #temp = corrs[((corrs<0.79) & (distances>0.5))]
    temp = corrs[(corrs<0.8)]
    #print(len(temp)==len(list_of_noun_sentences))
                          
    if (len(temp)==len(list_of_noun_sentences)):
        list_of_noun_sentences.append(noun_sentence_df_1000['noun_sentence'].values[idd])
        list_of_embeddings.append(noun_sentence_df_1000['noun_sentence_embeddings'].values[idd])
    else:
        if (len(corrs[(corrs>=0.8) & (corrs!=1)])>0):
            #print(list_of_noun_sentences)
            #print(list((corrs>=0.8) & (corrs!=1)))
            #print(list(np.array(list_of_noun_sentences)[(corrs>=0.8) & (corrs!=1)]))
            temp_list = noun_sentences_dictionary[list(np.array(list_of_noun_sentences)[(corrs>=0.8) & (corrs!=1)])[0]]
            #print(temp_list)
            temp_list.append(noun_sentence_df_1000['noun_sentence'].values[idd])              
            noun_sentences_dictionary[list(np.array(list_of_noun_sentences)[(corrs>=0.8) & (corrs!=1)])[0]] = temp_list         

In [ ]:
    

for idd in range(len(noun_sentence_df_50)):
#for idd in range(2):
    if (idd%10==0): print(idd,len(noun_sentence_df_50))
        
    count = 0
    corr = 0
    
    noun_sentence_df_50['noun_sentence_embeddings'].values.shape
    
    while ((corr < 0.8) & (corr!=1)):
        #print(noun_sentence_df_50['noun_sentence'].values[idd], ' -- ', list_of_noun_sentences[count])
        corr = pearsonr(noun_sentence_df_50['noun_sentence_embeddings'].values[idd],
                      list_of_embeddings[count])[0]
        
        count += 1
        
        #print(corr,count,len(list_of_embeddings))
        
        if (count==len(list_of_embeddings)):
            list_of_noun_sentences.append(noun_sentence_df_50['noun_sentence'].values[idd])
            list_of_embeddings.append(noun_sentence_df_50['noun_sentence_embeddings'].values[idd])
            print(idd,'new', noun_sentence_df_50['noun_sentence'].values[idd])
            break
        
    else:
        print(corr,count,len(list_of_embeddings))
        temp_list = noun_sentences_dictionary[list_of_noun_sentences[count-1]]
        temp_list.append(noun_sentence_df_50['noun_sentence'].values[idd])
        noun_sentences_dictionary[list_of_noun_sentences[count-1]] = temp_list 
    


In [ ]:
noun_sentences_dictionary

In [ ]:
print(noun_sentence_df_50['noun_sentence'].values[idd], ' -- ', list_of_noun_sentences[count-1])


In [ ]:
list_of_noun_sentences

In [ ]:
len(list_of_noun_sentences)

In [ ]:
['rm', 'ss', 'rec', 'fam', 'tlc', 'sep', 'hdwd', 'flrs', 'bd', 'ba', 'fr' ]

In [ ]:
    

for idd in range(len(noun_sentence_df_50)):
#for idd in range(50):
    if (idd%10==0): print(idd,len(noun_sentence_df_50))
        
    labelrefwords = noun_sentence_df_50['noun_sentence'].values[idd].split()
    corrs = np.zeros(len(list_of_noun_sentences))
    #distances = np.zeros(len(list_of_noun_sentences))
    #print(corrs)
    for jdd in range(len(list_of_noun_sentences)):
        labelwords = list_of_noun_sentences[jdd].split()
        corrs[jdd] = pearsonr(noun_sentence_df_50['noun_sentence_embeddings'].values[idd],
                      list_of_embeddings[jdd])[0]
        #distances[jdd] = model_gensim.wmdistance(labelrefwords, labelwords)
    #print(corrs)
    #print((corrs<0.79) & (distances>0.5))
    #temp = corrs[((corrs<0.79) & (distances>0.5))]
    temp = corrs[(corrs<0.8)]
    #print(len(temp)==len(list_of_noun_sentences))
                          
    if (len(temp)==len(list_of_noun_sentences)):
        list_of_noun_sentences.append(noun_sentence_df_50['noun_sentence'].values[idd])
        list_of_embeddings.append(noun_sentence_df_50['noun_sentence_embeddings'].values[idd])
    else:
        if (len(corrs[(corrs>=0.8) & (corrs!=1)])>0):
            #print(list_of_noun_sentences)
            #print(list((corrs>=0.8) & (corrs!=1)))
            #print(list(np.array(list_of_noun_sentences)[(corrs>=0.8) & (corrs!=1)]))
            temp_list = noun_sentences_dictionary[list(np.array(list_of_noun_sentences)[(corrs>=0.8) & (corrs!=1)])[0]]
            #print(temp_list)
            temp_list.append(noun_sentence_df_50['noun_sentence'].values[idd])              
            noun_sentences_dictionary[list(np.array(list_of_noun_sentences)[(corrs>=0.8) & (corrs!=1)])[0]] = temp_list   

In [ ]:
len(list_of_noun_sentences)

In [ ]:
list_of_noun_sentences

In [ ]:
noun_sentence_df_50['noun_sentence'].values[idd]

In [ ]:
noun_sentence_df_50[noun_sentence_df_50['noun_sentence']=='beautiful renovation']

In [ ]:
noun_sentence_df_50['noun_sentence']

In [ ]:
noun_sentence_df_1000[noun_sentence_df_1000['noun_sentence']=='beautiful renovation']

In [ ]:
print(noun_sentence_df_50['noun_sentence'].values[idd])

In [ ]:
list_of_noun_sentences

In [ ]:
list_of_noun_sentences[list_of_noun_sentences=='beautiful renovation']

In [ ]:
list(np.array(list_of_noun_sentences)[(corrs>=0.8) & (corrs!=1)])[0]

In [ ]:
noun_sentences_dictionary

In [ ]:

keys_list = list(noun_sentence_df_1000['noun_sentence'].values)
values_list = [[] for i in range(len(keys_list))]
noun_sentences_dictionary = dict(zip(keys_list, values_list))

In [ ]:
noun_sentences_dictionary

In [ ]:
len(list_of_noun_sentences)

In [ ]:
        #print(corr_temp, worddistance_tmp)
        if ((corr_temp<0.79) & (worddistance_tmp>0.5)):
            list_of_noun_sentences.append(noun_sentence_df_1000['noun_sentence'].values[idd])
            list_of_embeddings.append(noun_sentence_df_1000['noun_sentence_embeddings'].values[idd])

In [ ]:
list_of_noun_sentences

In [ ]:

def plot_similarity_serie_error(labelref, labels, errors, rotation):
    #print(labelref)
    #print(labels)
    message_embeddings_ = model(labels)
    print(message_embeddings_.shape)
    messageref_embeddings_ = model(labelref) 
    corr = np.zeros(len(labels))
    cossim = np.zeros(len(labels))
    worddistance = np.zeros(len(labels))
    
    labelrefwords = labelref[0].split()
    print(labelrefwords)
    
    for i in range(len(labels)):
        labelwords = labels[i].split()
        A = np.array(message_embeddings_[i,:])
        B = np.array(messageref_embeddings_)
        #print(A.shape,B.shape)
        corr[i] = pearsonr(A,B[0])[0]
        #cossim[i] = cosine_similarity(A.reshape(1,-1),B.reshape(1,-1))
        
        worddistance[i] = model_gensim.wmdistance(labelrefwords, labelwords)
        
    plt.plot(range(len(labels)),corr)
    plt.plot(range(len(labels)),worddistance)                                 
    
    for i, txt in enumerate(errors):
        plt.annotate(str(round(txt, 3)), xy =((range(len(labels)))[i], corr[i]), ha='center')
        
    plt.xticks(range(len(labels)),labels, rotation=rotation)
    plt.title(labelref)
    return corr, cossim, worddistance

In [ ]:

sentence_obama = 'Obama speaks to the media in Illinois'
sentence_president = 'The president greets the press in Chicago'

In [ ]:
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.
stop_words = stopwords.words('english')

def preprocess(sentence):
    return [w for w in sentence.lower().split() if w not in stop_words]

sentence_obama = preprocess(sentence_obama)
sentence_president = preprocess(sentence_president)

In [ ]:
sentence_obama

In [ ]:
sentence_obama.lower().split()

In [ ]:
distance = model_gensim.wmdistance(sentence_obama, sentence_president)
print('distance = %.4f' % distance)

In [ ]:
sentence_orange = preprocess('Oranges are my favorite fruit')
distance = model_gensim.wmdistance(sentence_obama, sentence_orange)
print('distance = %.4f' % distance)

In [ ]:
 model_gensim.wmdistance(['orange'], ['building'])